In [ ]:
%run models.ipynb
%run datasets.ipynb

In [ ]:
import dalex as dx
import numpy as np
import shap
import time
import signal

In [ ]:
svm = create_model('svm', kernel='linear')
xgboost = create_model('xgboost')

## Dobór porównywalnych czasów wykonania kernel shap i unbiased kernel shap

In [ ]:
X_train_svm, X_test_svm, y_train_svm, y_test_svm = get_test_train('house_price', train_size=200)
X_train_xgboost, X_test_xgboost, y_train_xgboost, y_test_xgboost = get_test_train('house_price', train_size=500)

In [ ]:
svm.fit(X_train_svm, y_train_svm)
svm.score(X_test_svm, y_test_svm)

In [ ]:
xgboost.fit(X_train_xgboost, y_train_xgboost)
xgboost.score(X_test_xgboost, y_test_xgboost)

In [ ]:
model_predicting_method = lambda m, d: m.predict(d)

In [ ]:
explainer_svm = dx.Explainer(svm, X_train_svm, y_train_svm, predict_function=model_predicting_method, \
                             label="SVM_house_price")
explainer_xgboost = dx.Explainer(xgboost, X_train_xgboost, y_train_xgboost, \
                                 predict_function=model_predicting_method, label="XGBoost_house_price")

kernel_explainer_svm = shap.KernelExplainer(svm.predict, X_train_svm)
kernel_explainer_xgboost = shap.KernelExplainer(xgboost.predict, X_train_xgboost)

In [ ]:
nb_of_features = len(X_test_svm.columns)

nb_of_test_samples = 500

def get_time_and_explanatinos(explainer, contributions, n_samples, model, is_dalex = True):
    time_list = []
    
    if model == "svm":
        X_test = X_test_svm
    else:
        X_test = X_test_xgboost
        
    for k in range(0, nb_of_test_samples):
        
        if is_dalex:
            start = time.time()
            
            explain = explainer.predict_parts(X_test.iloc[k:k+1], type="unbiased_kernel_shap",
                                              n_samples = n_samples, label=f"sample id: {k}")
            measured_time = time.time() - start
            contr = explain.result['contribution'].to_numpy()
        else:
            start = time.time()
            
            shapley_values = explainer.shap_values(X_test.iloc[k:k+1], nsamples = n_samples)
            measured_time = time.time() - start
            contr = shapley_values
        
        contributions[k, :] = contr
        time_list.append(measured_time)
    print(sum(time_list)/nb_of_test_samples)

In [ ]:
contributions_xgboost = np.zeros((nb_of_test_samples, nb_of_features))
get_time_and_explanatinos(explainer_xgboost, contributions_xgboost, 380, "xgboost")

In [ ]:
kernel_contributions_xgboost = np.zeros((nb_of_test_samples, nb_of_features))
get_time_and_explanatinos(kernel_explainer_xgboost, kernel_contributions_xgboost,  20, "xgboost", False)

In [ ]:
contributions_svm = np.zeros((nb_of_test_samples, nb_of_features))
get_time_and_explanatinos(explainer_svm, contributions_svm, 165, "svm")

In [ ]:
kernel_contributions_svm = np.zeros((nb_of_test_samples, nb_of_features))
get_time_and_explanatinos(kernel_explainer_svm, kernel_contributions_svm, 50, "svm", False)